In [5]:
from mpmath import *
from sympy import *

sym_x = Symbol('x')

mp.dps = 30
mp.prec = mp.dps * 3.34
mp.pretty = True

## takes [f a sympy function], [I the interval], [k an integer]
def midpt_integral(f, I, k): 
    sum = mpf(0)
    F = lambdify(sym_x, f, "mpmath")
    deltax = (I[1]-I[0])/mpf(k)
    for i in range(k):
        x = ((i/mpf(k))*I[1]) + ((1-(i/mpf(k)))*I[0]) + deltax/2
        sum += F(x)*deltax
    return sum

## [f a sympy function], [I the interval], [k an integer]
def threeeight_integral(f, I, k): 
    sum = mpf(0)
    F = lambdify(sym_x, f, "mpmath")
    deltax = (I[1]-I[0])/mpf(k)
    for i in range(k):
        xa = ((i/mpf(k))*I[1]) + ((1-i/mpf(k))*I[0]) 
        xb = xa + deltax/3
        xc = xb + deltax/3
        xd = xc + deltax/3
        sum += ( F(xa) + 3*F(xb) + 3*F(xc) + F(xd) )*deltax/8
    return sum

## takes [f a sympy function], [I the interval], [k an integer]
def spline_integral(f, I, k):
    fp = diff(f, sym_x)
    F = lambdify(sym_x, f, "mpmath")
    Fp = lambdify(sym_x, fp, "mpmath")
    ## the above allows us to call F and F'. 
    sum = mpf(0.0)
    deltax = (I[1]-I[0])/mpf(k)
    for i in range(k):
        xa = (i/float(k))*I[1] + (mpf(1.0)-(i/float(k)))*I[0]
        xb = xa + deltax
        
        c2 = -((deltax)**(-3)) * (3*(xb+xa)*(F(xa)-F(xb)+xb*Fp(xb)-xa*Fp(xa))-2*(xb**2+xb*xa+xa**2)*(Fp(xb)-Fp(xa)))
        c3 = -((deltax)**(-3)) * ( (-2*(F(xa)-F(xb)+xb*Fp(xb)-xa*Fp(xa))) + (xb+xa)*(Fp(xb)-Fp(xa)) )

        c1 = Fp(xa)-3*c3*(xa**2)-2*c2*xa
        c0 = F(xa) - c3*(xa**3) - c2*(xa**2) - c1*xa
        #print c0, c1, c2, c3
        sum += c0*(xb-xa) + (c1/2)*(xb**2 - xa**2) + (c2/3)*(xb**3-xa**3) + (c3/4)*(xb**4-xa**4)
    return sum

#sym_func = sym_x**3 ## 0 to 1 is 1/4
#print "Integral of x^3 from 0 to 1... should be 1/4."
#print "Midpt 30:   ", midpt_integral(sym_func, [mpf(0.0), mpf(1.0)], divisions)
#print "3/8 int 30: ", threeeight_integral(sym_func, [mpf(0.0), mpf(1.0)], divisions/3)
#print "Spline 30:  ", spline_integral(sym_func, [mpf(0.0), mpf(1.0)], divisions), "\n"

sym_func = sin(sym_x)
divisions = 10
print("Midpt ", divisions, ": ", midpt_integral(sym_func, [mpf(0.0), mpmath.pi], divisions))
print("3/8 int ", divisions, ": ", threeeight_integral(sym_func, [mpf(0.0), mpmath.pi], divisions//3))
print("Spline ", divisions, ": ", spline_integral(sym_func, [mpf(0.0), mpmath.pi], divisions), "\n")

divisions = 100
print("Midpt ", divisions, ": ", midpt_integral(sym_func, [mpf(0.0), mpmath.pi], divisions))
print("3/8 int ", divisions, ": ", threeeight_integral(sym_func, [mpf(0.0), mpmath.pi], divisions//3))
print("Spline ", divisions, ": ", spline_integral(sym_func, [mpf(0.0), mpmath.pi], divisions), "\n")

divisions = 1000
print("Midpt ", divisions, ": ", midpt_integral(sym_func, [mpf(0.0), mpmath.pi], divisions))
print("3/8 int ", divisions, ": ", threeeight_integral(sym_func, [mpf(0.0), mpmath.pi], divisions//3))
print("Spline ", divisions, ": ", spline_integral(sym_func, [mpf(0.0), mpmath.pi], divisions), "\n")


Midpt  10 :  2.0082484079079743546913938701
3/8 int  10 :  2.0003822420892664046832010172
Spline  10 :  1.9999728781779358213754882456 

Midpt  100 :  2.0000822490709862614159317284
3/8 int  100 :  2.0000000253572911782656840174
Spline  100 :  1.9999999972941315593390546046 

Midpt  1000 :  2.0000008224672703033269739693
3/8 int  1000 :  2.0000000000024448741303500186
Spline  1000 :  1.9999999999997201996573554638 



In [6]:
## So it appears generally the spline method is a tiny bit more accurage than the 3/8 method.  But they appear to
## be on roughly the same order of magnitude.  For comparison's sake, let's check the relative run-times. 

import time

timesM = []
for i in range(5, 10):
    startT = time.time()
    midpt_integral(sym_func, [mpf(0.0), mpf(1.0)], 30*i*i)
    endT = time.time()
    timesM.append(endT-startT)
print("Midpoint times: ", timesM)

timesM = []
for i in range(5, 10):
    startT = time.time()
    threeeight_integral(sym_func, [mpf(0.0), mpf(1.0)], 10*i*i)
    endT = time.time()
    timesM.append(endT-startT)
print("3/8 times: ", timesM)

timesM = []
for i in range(5, 10):
    startT = time.time()
    spline_integral(sym_func, [mpf(0.0), mpf(1.0)], 30*i*i)
    endT = time.time()
    timesM.append(endT-startT)
print("spline times: ", timesM)

Midpoint times:  [0.03927421569824219, 0.04446244239807129, 0.057367801666259766, 0.07830286026000977, 0.09740138053894043]
3/8 times:  [0.025030136108398438, 0.03654146194458008, 0.04918646812438965, 0.06427812576293945, 0.07941818237304688]
spline times:  [0.2687711715698242, 0.38745665550231934, 0.5250518321990967, 0.680504322052002, 0.8696763515472412]


From this perspective, the 3/8-ths technique is the fastest, the spline **by far** the slowest.  So perhaps 3/8
Would make more sense to use.  Perhaps we can simplify the algebra in the spline technique? We can probably factor-out some of the xb-xa = deltax terms. It looks like the algebra is more computation-intensive than it needs to be.  Well, I'm not having any luck with that. 